**Ejercicio 1. Manipulacion de datos**

In [27]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Configuración
num_empleados = 50
departamentos = ["Ventas", "TI", "Recursos Humanos", "Finanzas", "Operaciones"]
cargos = ["Analista", "Gerente", "Asistente", "Especialista", "Director"]

# 1. Tabla: Empleados (Maestro)
empleados = []
for i in range(1, num_empleados + 1):
    id_emp = f"E{i:03d}"
    nombre = f"Empleado_{i}"
    depto = random.choice(departamentos)
    cargo = random.choice(cargos)
    salario = random.randint(800, 3500)
    fecha_ingreso = datetime(2018, 1, 1) + timedelta(days=random.randint(0, 2000))
    genero = random.choice(["M", "F"])
    empleados.append([id_emp, nombre, depto, cargo, salario, fecha_ingreso.strftime("%Y-%m-%d"), genero])

df_empleados = pd.DataFrame(empleados, columns=["ID_Empleado", "Nombre", "Departamento", "Cargo", "Salario_Mensual", "Fecha_Ingreso", "Genero"])

# 2. Tabla: Evaluaciones de Desempeño (Hechos - Anual)
# Asumimos la evaluación del último año
evaluaciones = []
for emp in empleados:
    id_emp = emp[0]
    # Score 1 to 5
    score = random.choices([1, 2, 3, 4, 5], weights=[5, 10, 40, 35, 10])[0] # Weighted towards 3 and 4
    fecha_eval = "2023-12-15"
    evaluaciones.append([id_emp, fecha_eval, score])

df_evaluaciones = pd.DataFrame(evaluaciones, columns=["ID_Empleado", "Fecha_Evaluacion", "Puntaje_Desempeno"])

# 3. Tabla: Registro de Asistencia (Hechos - Mensual Resumido)
# Datos de los últimos 3 meses para cada empleado
asistencia = []
meses = ["2024-01", "2024-02", "2024-03"]

for emp in empleados:
    id_emp = emp[0]
    for mes in meses:
        dias_ausente = random.choices([0, 1, 2, 3], weights=[70, 15, 10, 5])[0]
        horas_extra = random.randint(0, 10)
        asistencia.append([id_emp, mes, dias_ausente, horas_extra])

df_asistencia = pd.DataFrame(asistencia, columns=["ID_Empleado", "Mes", "Dias_Ausente", "Horas_Extra"])

In [28]:
print("Empleados Head:")
print(df_empleados.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nEvaluaciones Head:")
print(df_evaluaciones.head().to_markdown(index=False, numalign="left", stralign="left"))
print("\nAsistencia Head:")
print(df_asistencia.head().to_markdown(index=False, numalign="left", stralign="left"))

Empleados Head:
| ID_Empleado   | Nombre     | Departamento     | Cargo        | Salario_Mensual   | Fecha_Ingreso   | Genero   |
|:--------------|:-----------|:-----------------|:-------------|:------------------|:----------------|:---------|
| E001          | Empleado_1 | Ventas           | Director     | 1297              | 2021-09-19      | F        |
| E002          | Empleado_2 | Ventas           | Asistente    | 801               | 2018-08-22      | M        |
| E003          | Empleado_3 | TI               | Asistente    | 3124              | 2022-11-18      | M        |
| E004          | Empleado_4 | Operaciones      | Gerente      | 1049              | 2023-03-07      | F        |
| E005          | Empleado_5 | Recursos Humanos | Especialista | 1737              | 2020-10-02      | M        |

Evaluaciones Head:
| ID_Empleado   | Fecha_Evaluacion   | Puntaje_Desempeno   |
|:--------------|:-------------------|:--------------------|
| E001          | 2023-12-15         | 4    

In [29]:
n_by_state = df_asistencia.groupby("Dias_Ausente")["ID_Empleado"].count()
print(n_by_state)


Dias_Ausente
0    106
1     23
2     16
3      5
Name: ID_Empleado, dtype: int64


Union de los datos en las tablas

In [30]:
df_tabla_conjunta2 = pd.merge(df_empleados,df_asistencia,on="ID_Empleado",how="inner")

print(df_tabla_conjunta2)


    ID_Empleado       Nombre Departamento         Cargo  Salario_Mensual  \
0          E001   Empleado_1       Ventas      Director             1297   
1          E001   Empleado_1       Ventas      Director             1297   
2          E001   Empleado_1       Ventas      Director             1297   
3          E002   Empleado_2       Ventas     Asistente              801   
4          E002   Empleado_2       Ventas     Asistente              801   
..          ...          ...          ...           ...              ...   
145        E049  Empleado_49           TI      Analista             1788   
146        E049  Empleado_49           TI      Analista             1788   
147        E050  Empleado_50     Finanzas  Especialista             1089   
148        E050  Empleado_50     Finanzas  Especialista             1089   
149        E050  Empleado_50     Finanzas  Especialista             1089   

    Fecha_Ingreso Genero      Mes  Dias_Ausente  Horas_Extra  
0      2021-09-19      F

In [31]:
#Prom. dias ausencia
prom_ausencias = (df_asistencia.groupby("ID_Empleado")["Dias_Ausente"].mean().reset_index().rename(columns={"Dias_Ausente": "Promedio_Dias_Ausente"}))

#Prom. horas extra
prom_horas_extra = (df_asistencia.groupby("ID_Empleado")["Horas_Extra"].mean().reset_index())

#Metodo Merge
df_base = pd.merge(df_empleados,df_evaluaciones,on="ID_Empleado",how="inner")
df_base = pd.merge(df_base,prom_ausencias,on="ID_Empleado",how="inner")
df_base = pd.merge(df_base,prom_horas_extra,on="ID_Empleado",how="inner")

df_base.head()


,ID_Empleado,Nombre,Departamento,Cargo,Salario_Mensual,Fecha_Ingreso,Genero,Fecha_Evaluacion,Puntaje_Desempeno,Promedio_Dias_Ausente,Horas_Extra
0,E001,Empleado_1,Ventas,Director,1297,2021-09-19,F,2023-12-15,4,0.666667,5.333333
1,E002,Empleado_2,Ventas,Asistente,801,2018-08-22,M,2023-12-15,3,0.666667,3.000000
2,E003,Empleado_3,TI,Asistente,3124,2022-11-18,M,2023-12-15,4,0.000000,6.666667
3,E004,Empleado_4,Operaciones,Gerente,1049,2023-03-07,F,2023-12-15,4,0.333333,2.333333
4,E005,Empleado_5,Recursos Humanos,Especialista,1737,2020-10-02,M,2023-12-15,4,0.000000,2.333333


**Pregunta 1: ¿Cuántos empleados cumplen esto? ¿Quiénes son?**

In [32]:
empleados_estrella = df_base[(df_base["Puntaje_Desempeno"]>=4)&(df_base["Promedio_Dias_Ausente"]<1)]
empleados_estrella[["ID_Empleado", "Nombre", "Departamento","Puntaje_Desempeno", "Promedio_Dias_Ausente"]]


,ID_Empleado,Nombre,Departamento,Puntaje_Desempeno,Promedio_Dias_Ausente
0,E001,Empleado_1,Ventas,4,0.666667
2,E003,Empleado_3,TI,4,0.000000
3,E004,Empleado_4,Operaciones,4,0.333333
4,E005,Empleado_5,Recursos Humanos,4,0.000000
5,E006,Empleado_6,Operaciones,4,0.333333
6,E007,Empleado_7,TI,4,0.000000
7,E008,Empleado_8,Finanzas,5,0.000000
9,E010,Empleado_10,TI,4,0.000000
10,E011,Empleado_11,Operaciones,5,0.333333
11,E012,Empleado_12,Ventas,4,0.333333


In [33]:
empleados_estrella = df_base[(df_base["Puntaje_Desempeno"]>=4)&(df_base["Promedio_Dias_Ausente"]<1)]
empleados_estrella[["ID_Empleado", "Nombre", "Departamento","Puntaje_Desempeno", "Promedio_Dias_Ausente"]]

#Cantidad total de empleados estrella
cantidad_estrella_df = empleados_estrella[["ID_Empleado"]].count()
print("Cantidad de empleados estrella:")
print(cantidad_estrella_df)



Cantidad de empleados estrella:
ID_Empleado    23
dtype: int64


* 18 empleados cumplen con esto, son aquellos que se pueden visualizar en la tabla superior.

**Pregunta 2: ¿Existe algún empleado con salario alto (ej. > $2000) que tenga bajo desempeño?**

In [34]:
df_salarios = df_base.sort_values(by="Salario_Mensual",ascending=False)
empleados_incoherentes = df_salarios[(df_salarios["Puntaje_Desempeno"]<=2)&(df_salarios["Salario_Mensual"]>2000)]
empleados_incoherentes[["ID_Empleado", "Nombre", "Salario_Mensual", "Puntaje_Desempeno"]]


,ID_Empleado,Nombre,Salario_Mensual,Puntaje_Desempeno
23,E024,Empleado_24,2984,2
25,E026,Empleado_26,2812,2
31,E032,Empleado_32,2231,1


* Si existen, esto se confirma ya que en la tabla superior se presentan empleados que cumplen con los requisitos que nos permiten determinar eso.